# 重回帰分析

目的変数$y$を説明変数$x_1, x_2, x_3, ..., x_d$の線形関数と定数項で説明するモデルのこと。
$$
y = B_0 + B_1x_1 + ... + B_dx_d + ε
$$
ただし、$B_1, B_2, ..., B_d\in{R}$は各変数の$y$への影響を表現する**回帰係数**で、$B_0$は切片項にあたる。  
また、$ε$は誤差項で平均0かつ分散$σ^2$とする。

In [27]:
# wineデータセットで実践
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

wine_data = load_wine()
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

wine_df_train, wine_df_test = train_test_split(wine_df, train_size=100)

x_train = wine_df_train.drop('alcohol', axis=1) # 説明変数x1, x2, x3,... ,xd
y_train = wine_df_train['alcohol'] # 目的変数y

x_test = wine_df_test.drop('alcohol', axis=1)
y_test = wine_df_test['alcohol']

model = LinearRegression()
model.fit(x_train, y_train)
pred_y = model.predict(x_test)

print(f"予測結果：{[round(pred_y[n], 2) for n in range(len(pred_y))]}")
print(f"実際の値：{y_test.to_list()}")


予測結果：[12.83, 12.07, 12.61, 12.41, 13.95, 13.31, 11.94, 12.43, 12.45, 12.7, 12.31, 12.98, 13.73, 14.56, 12.71, 13.47, 13.35, 12.54, 12.83, 12.98, 13.62, 12.53, 13.31, 13.57, 13.28, 12.15, 12.69, 13.67, 12.8, 12.91, 13.53, 13.07, 12.77, 13.65, 12.01, 13.38, 12.21, 14.14, 13.33, 13.69, 12.64, 12.93, 13.04, 13.94, 13.67, 12.49, 12.77, 12.22, 12.12, 12.73, 12.95, 12.22, 14.1, 12.66, 13.26, 13.56, 12.56, 12.54, 12.63, 13.87, 12.47, 12.57, 13.51, 12.61, 12.51, 13.36, 13.32, 13.45, 12.31, 13.19, 13.35, 12.48, 12.05, 12.42, 12.91, 12.21, 13.37, 12.29]
実際の値：[12.53, 12.37, 13.88, 12.25, 14.21, 13.32, 12.08, 12.77, 11.64, 12.88, 11.84, 13.68, 14.23, 14.19, 11.87, 13.56, 13.48, 12.37, 12.85, 11.96, 13.05, 12.07, 11.56, 13.75, 14.75, 12.42, 12.93, 13.76, 12.7, 11.45, 13.41, 13.36, 11.46, 13.9, 12.0, 13.71, 11.79, 14.38, 14.83, 12.25, 12.81, 12.96, 13.24, 13.27, 14.39, 11.84, 13.11, 12.29, 12.0, 12.2, 13.69, 12.6, 13.58, 12.34, 13.84, 13.71, 13.52, 12.43, 13.05, 14.06, 11.62, 13.34, 13.83, 12.21, 11.

目的変数の変動を説明変数で説明できる部分とそれ以外の部分に分けることで、モデルの説明力を調べることができる。  
**決定係数**$R^2$はこの考えを反映させたもので、
$$
R^2 = \frac{\sum_{i=1}^{n} ((x_i-\bar{x})^T \hat{\beta}_{1:d})^2}{\sum_{i=1}^{n} (y_i-\bar{y})^2} = \
1-\frac{\sum_{i=1}^{n} (y_i-(1,x_i^T)^T \hat{\beta})^2}{\sum_{i=1}^{n} (y_i-\bar{y})^2}
$$
と定義される。要するに「$1-($回帰残差$)^2$の総和$/($偏差$)^2$の総和」である。決定係数が大きければそれだけデータのへの当てはまりが良いことを意味する。  
なお、決定係数は変数を増やせば増やすほど増大するので、変数の数について調整した**自由度調整済み決定係数**を用いることが多い。  
自由度調整済み決定係数$R^{*2}$は、
$$
R^{*2} = \frac{\sum_{i=1}^{n} (y_i-(1,x_i^T)^T \hat{\beta})^2/(n-d-1)}{\sum_{i=1}^{n} (y_i-\bar{y})^2/(n-1)} = \
1-(1-R^2)\frac{n-1}{n-d-1}
$$
で定義される。

In [28]:
# 決定係数と自由度調整済み決定係数を算出
from sklearn.metrics import r2_score

r2 = r2_score(y_test, pred_y)
print(f"決定係数(r2):{round(r2,3)}")

def adj_r2_score(y_true, y_pred, n_dim):
    y_true_list = y_true.tolist()
    y_pred_list = y_pred.tolist()
    data_size = len(y_true_list)

    ysos = 0
    rsos = 0
    y_mean = sum(y_true_list)/len(y_true_list)
    for i in range(data_size):
        ysos += pow(y_true_list[i]-y_mean, 2)
        residual = y_true_list[i] - y_pred_list[i]
        rsos += pow(residual, 2)
    return 1-(rsos/(data_size-n_dim-1))/(ysos/(data_size-1))

adj_r2 = adj_r2_score(y_test, pred_y, len(x_test.columns))
print(f"自由度調整済み決定係数(r2):{round(adj_r2,3)}")

決定係数(r2):0.448
自由度調整済み決定係数(r2):0.346


# 重回帰分析の検定

重回帰分析においては、変数の**有意性検定**が重要である。  
たとえば、回帰変数$B_k$がゼロか非ゼロかを検定することにより、変数$x_k$が$y$に有意に影響しているかどうかがわかる。  
これは推定結果の解釈に重要な示唆を与え、データ分析における強力な解析手法となりうる。  
変数の有意性検定を一般化すると、以下のように記述できる。  
ある$q<d+1$を満たす$q$に対し、行列$A \in R^{q(d+1)}$は、ランクが$rank(A)=q$かつ像が$Im(A^T) \subset Im(X^T)$を満たすとする。  
このAを用いて、
$$
帰無仮説H_0：A\beta=0、\quad v.s \quad
対立仮説H_1：A\beta≠0
$$
として書ける仮説検定を考える。  
たとえば、$A=(0,0,...,1,0,..0)$のように$k+1$番目の成分のみが1のベクトルを考えれば、$A\beta=0 \leftrightarrow \beta_k=0$となる。  
このとき、$Θ_0=\{\beta \in \mathbb{R}^{d+1} | A\beta=0\}$として、
$$
R_0^2 = \min_{\beta \in Θ_0}||Y-X\beta||^2
$$
とおき、
$$
R_1^2 = \min_{\beta \in \mathbb{R}^{d+1}}||Y-X\beta||^2
$$
とおく。  
$R_0^2$は帰無仮説のもとで二乗損失がどれだけ小さくできるかを表し、$R_1^2$は対立仮説のもとで二乗損失がどれだけ小さくできるかを表す。  
なお$R_1^2=||e||^2$であることには注意。  
もし、$R_0^2$と$R_1^2$が大きく変わる場合は、帰無仮説は誤っているとして棄却すればよい。  
そこで統計検定量として、
$$
T = \frac{(R_0^2-R_1^2)/q}{R_1^2/(n-d-1)}
$$
を考える。すると帰無仮説のもと、
$$
T \sim F(q, n-d-1)
$$
が成り立つ。なおTは統計量である

In [29]:
# 回帰係数の検定を実行
# A=(0,0,...,1,0,..0)(k+1番目の成分のみが1)とする
import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_test, sm.add_constant(x_test))
result = model.fit()
A = np.identity(len(result.params))
A = A[1:,:]
result.f_test(A)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=8.694660789609067, p=1.3977043107417128e-09, df_denom=65, df_num=12>

In [30]:
# 定数項の検定を実行
# A=(1,0,...,0,0,..0)(1番目の成分のみが1)とする
A = np.identity(len(result.params))
A = A[0,:]
result.f_test(A)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=130.0730454268503, p=3.6889966903540655e-17, df_denom=65, df_num=1>

# 正則化

重回帰分析では$X^TX$が可逆であることを仮定していたが、高次元データを扱う場合は必ずしもそうなるとは限らない。  
そこで$X^TX$の条件数が悪い場合でも安定した推定を行うために**正則化**が有用である。  
代表的な正則化手法としては、L1正則化とL2正則化がある。  
L1正則化を用いた重回帰はLasso回帰、L2正則化を用いた重回帰はRidge回帰ともよばれる。  
正則化を加えることで推定の分散を抑えることができる。  
そのため、**学習データ**に合わせすぎて予測精度が悪くなる**過適合**を抑止することができる。  
一方で正則化が強すぎると推定量が縮小しすぎて、予測精度が悪化する**過小適合**が発生する点に注意

L1正則化(Lasso回帰)では、正則化項としてのL1ノルムを使用。L1ノルムはベクトル成分の絶対値の和(マンハッタン距離と呼ばれる)である。  
回帰係数$\beta$のいくつかの成分は0(=**スパース**)になり、凸最適化によって変数選択が可能になる。

In [31]:
# Lasso回帰を実行
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.1)

# モデル学習と予測
model.fit(x_train, y_train)
pred_y = model.predict(x_test)

lasso_result_df = pd.DataFrame({
    "variable":x_train.columns,
    "coef":model.coef_
})
print("回帰係数")
print(lasso_result_df.to_string(index=False))

print(f"決定係数(r2):{round(r2_score(y_test, pred_y),3)}")
print(f"自由度調整済み決定係数(r2):{round(adj_r2_score(y_test, pred_y, len(x_test.columns)),3)}")

回帰係数
                    variable      coef
                  malic_acid  0.027706
                         ash  0.000000
           alcalinity_of_ash -0.000000
                   magnesium  0.002435
               total_phenols  0.000000
                  flavanoids  0.000000
        nonflavanoid_phenols -0.000000
             proanthocyanins  0.000000
             color_intensity  0.112084
                         hue -0.000000
od280/od315_of_diluted_wines  0.000000
                     proline  0.001318
決定係数(r2):0.49


TypeError: 'float' object is not callable

L2正則化(Ridge回帰)では、正則化項としてのL2ノルムを使用。L2ノルムはベクトル成分の絶対値の和(ユークリッド距離と呼ばれる)である。  
正則化パラメータを洗濯する基準として**Mallows' CP基準**がある

In [ ]:
# Ridge回帰を実行
from sklearn.linear_model import Ridge

model = Ridge(alpha=1.0)

# モデル学習と予測
model.fit(x_train, y_train)
pred_y = model.predict(x_test)

ridge_result_df = pd.DataFrame({
    "variable":x_train.columns,
    "coef":model.coef_
})

print("回帰係数")
print(ridge_result_df.to_string(index=False))

print(f"決定係数(r2):{round(r2_score(y_test, pred_y),3)}")
print(f"自由度調整済み決定係数(r2):{round(adj_r2_score(y_test, pred_y, len(x_test.columns)),3)}")

回帰係数
                    variable      coef
                  malic_acid  0.135406
                         ash -0.002799
           alcalinity_of_ash -0.030554
                   magnesium  0.002728
               total_phenols -0.035880
                  flavanoids  0.157312
        nonflavanoid_phenols  0.151086
             proanthocyanins -0.147887
             color_intensity  0.147098
                         hue -0.034982
od280/od315_of_diluted_wines  0.105350
                     proline  0.001052
決定係数(r2):0.442
自由度調整済み決定係数(r2):-0.005


L1正則化とL2正則化を混ぜた**Elastic-Net**と呼ばれる手法もある。  
2つの相関が強い変数があるとその2つの変数間で変数選択が安定しないL1正則化の欠点を克服。相関の高い両方の変数を用いる

In [ ]:
# Elastic-Netを実行
from sklearn.linear_model import ElasticNet

model = ElasticNet(alpha=1.0, l1_ratio=0.5) # l1_ratioはL1ノルムとL2ノルムの比率

# モデル学習と予測
model.fit(x_train, y_train)
pred_y = model.predict(x_test)

eln_result_df = pd.DataFrame({
    "variable":x_train.columns,
    "coef":model.coef_
})

print("回帰係数")
print(eln_result_df.to_string(index=False))

print(f"決定係数(r2):{round(r2_score(y_test, pred_y),3)}")
print(f"自由度調整済み決定係数(r2):{round(adj_r2_score(y_test, pred_y, len(x_test.columns)),3)}")

回帰係数
                    variable      coef
                  malic_acid  0.000000
                         ash  0.000000
           alcalinity_of_ash -0.000000
                   magnesium  0.000000
               total_phenols -0.000000
                  flavanoids -0.000000
        nonflavanoid_phenols  0.000000
             proanthocyanins -0.000000
             color_intensity  0.020215
                         hue -0.000000
od280/od315_of_diluted_wines -0.000000
                     proline  0.001610
決定係数(r2):0.299
自由度調整済み決定係数(r2):-0.261
